# Geocode the EUC dataframe

In [1]:
import pandas as pd
import numpy as np

In [88]:
# API key is stored separately due to security concerns (google_api.key is in gitignore)
file1 = open("google_api.key","r")
API_KEY_ = file1.read().rstrip()
file1.close()

In [2]:
import googlemaps
gmaps = googlemaps.Client(key=API_KEY)

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="VI13")

In [3]:
df_euc = pd.read_excel("datasets/EUC_ for_the_academic_year_2013_2014.xls")
df = df_euc.sample(100)

In [45]:
import time

def google_search(institution, city, country):
    geocode_result = gmaps.geocode(institution + ", " + city + ", " + country)
    if len(geocode_result) > 0:
        try:
            city_name = geocode_result[0]["address_components"][2]["long_name"]
        except:
            city_name = None
        coordinates = tuple(geocode_result[0]["geometry"]["location"].values())
        return tuple(list(["org", city_name, coordinates]))
    
    geocode_result = gmaps.geocode(city + ", " + country)
    if len(geocode_result) > 0:
        city_name = None
        try:
            city_name = geocode_result[0]["address_components"][2]["long_name"]
        except:
            city_name = None
        coordinates = tuple(geocode_result[0]["geometry"]["location"].values())
        return tuple(list(["cit", city_name, coordinates]))
    return None

In [46]:
df_euc["query"] = df_euc.apply(lambda x: google_search(x["Name of Organisation"], x["City"], x["Country"]), axis=1)

In [47]:
df_euc.to_csv("datasets/EUC_google_geocoded.csv", index=False)

In [59]:
df_euc["coordinates"] = df_euc["query"].apply(lambda x: x[2])
df_euc["coord_type"] = df_euc["query"].apply(lambda x: x[0])
df_euc["coord_city"] = df_euc["query"].apply(lambda x: x[1])

In [69]:
euc_clean = df_euc[["Institutional code", "Name of Organisation", "Country", "City", "coordinates"]]

In [ ]:
# Are there any duplicates?
euc_clean["Institutional code"].duplicated().value_counts()

In [75]:
euc_clean = euc_clean.to_csv("datasets/EUC_geocoded_clean.csv", index=False)

In [79]:
euc_clean = pd.read_csv("datasets/EUC_geocoded_clean.csv")

In [80]:
euc_clean

,Institutional code,Name of Organisation,Country,City,coordinates
0,A BADEN01,PAEDAGOGISCHE HOCHSCHULE NIEDEROESTERREICH,AT,Baden,"(48.00849239999999, 16.2552558)"
1,A DORNBIR01,FACHHOCHSCHULE VORARLBERG GMBH,AT,Dornbirn,"(47.4067642, 9.744422300000002)"
2,A EISENST01,STIFTUNG PRIVATE PÄDAGOGISCHE HOCHSCHULE BURGE...,AT,Eisenstadt,"(47.8295268, 16.5337338)"
3,A EISENST02,Fachhochschulstudiengaenge Burgenland Gesellsc...,AT,Eisenstadt,"(47.8294118, 16.5350525)"
4,A EISENST05,JOSEPH HAYDN KONSERVATORIUM DES LANDES BURGENLAND,AT,Eisenstadt,"(47.8500376, 16.514385)"
...,...,...,...,...,...
4914,UK WOLVERH01,UNIVERSITY OF WOLVERHAMPTON,UK,WOLVERHAMPTON,"(52.58817699999999, -2.1274393)"
4915,UK WORCEST01,UNIVERSITY OF WORCESTER,UK,Worcester,"(52.1970053, -2.2428253)"
4916,UK YORK01,UNIVERSITY OF YORK,UK,YORK,"(53.9626623, -1.0864513)"
4917,UK YORK03,YORK ST JOHN UNIVERSITY,UK,YORK,"(53.9670826, -1.0821198)"
